# Usage of GC.Analysis.API for GC Analysis

In [ ]:
#r "nuget: Microsoft.Diagnostics.Tracing.TraceEvent, 3.1.9"
#r "nuget: XPlot.Plotly"
#r "nuget: XPlot.Plotly.Interactive"
#r "nuget: Microsoft.Data.Analysis"
#r "nuget: Newtonsoft.Json"

using Etlx = Microsoft.Diagnostics.Tracing.Etlx;
using Microsoft.Data.Analysis;
using Microsoft.Diagnostics.Tracing.Analysis.GC;
using Microsoft.Diagnostics.Tracing.Analysis;
using Microsoft.Diagnostics.Tracing.Parsers.Clr;
using Microsoft.Diagnostics.Tracing;
using XPlot.Plotly;

using System.IO;
using Newtonsoft.Json;

## Building and Using The GC Analysis API

In [ ]:
dotnet build -c Release "..\..\GC.Analysis.API"

In [ ]:
#r "C:\performance\artifacts\bin\GC.Analysis.API\Release\net8.0\GC.Analysis.API.dll" 

using GC.Analysis.API;

## Creating the Analyzer

In [ ]:
var TRACE_PATH  = @".\Traces\vs-gccollectonly.etl.zip";
var BASE_PATH   = @".\Traces\";

### Get All Analyzers From Multiple Trace Paths

In [ ]:
Dictionary<string, Analyzer> gcTraceData = AnalyzerManager.GetAnalyzer(tracePaths: new[] { TRACE_PATH });

### Get All Analyzers From a Base Path

In [ ]:
Dictionary<string, Analyzer> gcTraceData = AnalyzerManager.GetAllAnalyzers(BASE_PATH);

### Get Analyzer From a Single Path

#### Get All Processes From a Trace

In [ ]:
Analyzer gcTraceData = new Analyzer(tracePath: TRACE_PATH);

#### Get Select Processes From a Trace

In [ ]:
Analyzer gcTraceData = new Analyzer(tracePath: TRACE_PATH, processNames: new HashSet<string> { "devenv" });

## Trace Summarization

In [ ]:
gcTraceData.SummarizeTrace(processName: "devenv")

## GC Analysis

### GC Summary

#### Summarize By Process Name

In [ ]:
gcTraceData.Summarize(processName: "devenv")

#### Comparative Summary

In [ ]:
List<GCProcessData> allDevenvs = gcTraceData.GetProcessGCData("devenv");
var data = allDevenvs[0].Compare(new [] { allDevenvs[1], allDevenvs[2] });
data

#### Summarize Based On All Processes Sorted By Some Criteria 

In [ ]:
gcTraceData.Summarize(topN: 3, criteriaInGCStats: nameof(GCStats.MeanSizeAfterMB))

### Charting

#### Getting To a ``List<TraceGC>`` from the Trace Data For a Process

In [ ]:
List<GCProcessData> devenvChartData = gcTraceData.GetProcessGCData(processName: "devenv");
GCProcessData devenvToInvestigate   = devenvChartData[0];
List<TraceGC> devenvTraceGCs        = devenvToInvestigate.GCs;

#### Charting Single Series

In [ ]:
GCCharting.ChartGCData(gcs       : devenvTraceGCs, 
                       title     : "Pause Duration (MSec)", 
                       fieldName : nameof(TraceGC.PauseDurationMSec), 
                       xAxis     : nameof(TraceGC.Number))

#### Charting Single Series with Chart Info

In [ ]:
ChartInfo chartInfo = new ChartInfo
{
    YAxisLabel = "MSec",
    XAxisLabel = "GC #",
    Width = 1000,
    Height = 500,
};

GCCharting.ChartGCData(gcs       : devenvTraceGCs, 
                       title     : "Pause Duration (MSec)", 
                       fieldName : nameof(TraceGC.PauseDurationMSec), 
                       xAxis     : nameof(TraceGC.Number),
                       chartInfo : chartInfo)

#### Charting Multiple Series From One List of Trace GCs

In [ ]:
GCCharting.ChartGCData(gcs        : devenvTraceGCs,
                       title      : "Pause Durations and Suspend Durations",
                       fields     : new [] { ( "Pause Duration (MSec)", nameof(TraceGC.PauseDurationMSec)),
                                           ( "Suspend Duration (MSec)", nameof(TraceGC.SuspendDurationMSec)) },
                       xAxis      : nameof(TraceGC.Number),
                       chartInfo  : chartInfo)

#### Charting Multiple Series From Different Trace GCs

##### Non Relative GCs

In [ ]:
GCProcessData otherDevenvToInvestigate = devenvChartData[1];
List<TraceGC> devenvTraceGCsOther      = otherDevenvToInvestigate.GCs;

List<(string scatterName, List<TraceGC> gcs)> gcData = 
    new()
    {
        { ( scatterName :  "Devenv 0 - Pause Duration (MSec)" , gcs : devenvTraceGCs )},
        { ( scatterName :  "Devenv 1 - Pause Duration (MSec)" , gcs : devenvTraceGCsOther )}
    };

GCCharting.ChartGCData(gcData          : gcData, 
                       title           : "Pause Duration Comparisons Between Devenvs", 
                       isXAxisRelative : false,
                       fieldName       : nameof(TraceGC.PauseDurationMSec))

##### Relative GCs

In [ ]:
List<(string scatterName, List<TraceGC> gcs)> gcData = 
    new()
    {
        { ( scatterName :  "Devenv 0 - Pause Duration (MSec)" , gcs : devenvTraceGCs )},
        { ( scatterName :  "Devenv 1 - Pause Duration (MSec)" , gcs : allDevenvs[1].GCs )}
    };

GCCharting.ChartGCData(gcData          : gcData, 
                       title           : "Pause Duration Comparisons Between Devenvs", 
                       fieldName       : nameof(TraceGC.PauseDurationMSec),
                       isXAxisRelative : true,
                       xAxis           : nameof(TraceGC.Number))

#### Charting Multiple Series With Filters

In [ ]:
IEnumerable<(string, Func<TraceGC, bool>)> filters = new (string, Func<TraceGC, bool>)[] 
{
    ("Generation0", (TraceGC gc) => gc.Generation == 0),
    ("Generation1", (TraceGC gc) => gc.Generation == 1),
};

GCCharting.ChartGCData(gcs       : devenvTraceGCs, 
                       title     : "Per Generation Pause Duration (MSec)", 
                       fieldName : nameof(TraceGC.PauseDurationMSec), 
                       filters   : filters)

#### Charting Custom Objects

In [ ]:
class CustomType
{
    public double Value0;
    public double Value1;
    public int Index;
}

var NUM_GCs = 5;

CustomType[] customTypes = new CustomType[NUM_GCs];
for(int customTypeIdx = 0; customTypeIdx < customTypes.Length; customTypeIdx++)
{
    customTypes[customTypeIdx] = new CustomType();
    var gc = devenvTraceGCs[customTypeIdx];

    customTypes[customTypeIdx].Value0 = gc.HeapSizeAfterMB; 
    customTypes[customTypeIdx].Value1 = gc.HeapSizeBeforeMB;
    customTypes[customTypeIdx].Index  = customTypeIdx;
}

GCCharting.ChartGCData(gcData : customTypes, 
                       title  : "Charting Custom Data",
                       fields : new [] { ( "Value0", nameof(CustomType.Value0) ), 
                                       ( "Value1", nameof(CustomType.Value1) )},
                       xAxis  : nameof(CustomType.Index))

## Debugging

In [ ]:
Console.WriteLine($"Current Process ID: {System.Diagnostics.Process.GetCurrentProcess().Id}");

#!about